In [ ]:
# Import the required packages.

# Need to install TEEHR to avoid this
import sys
sys.path.insert(0, "../../src")

import teehr.loading.nwm_grid_data as tlg

In [ ]:
# Set some notebook variables to point to the relevant study files.
RUN = "medium_range_mem1"  # analysis_assim, short_range, analysis_assim_hawaii, medium_range_mem1
OUTPUT_TYPE = "channel_rt_1"
VARIABLE_NAME = "streamflow"

START_DATE = "2023-03-18"
INGEST_DAYS = 10

JSON_DIR = '/home/jovyan/temp/parquet/jsons/'
OUTPUT_DIR = "/home/jovyan/temp/parquet"